In [1]:
#!pip install python-dotenv openai

In [11]:
import os
import pandas as pd
from time import sleep
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

# Initialize client pointing to Google PaLM
client = OpenAI(
    api_key=GOOGLE_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Load the CSV
df = pd.read_csv("output/chinese_title.csv")

def batch_titles(titles, batch_size=10):
    for i in range(0, len(titles), batch_size):
        yield titles[i:i + batch_size]

def translate_titles(titles_batch):
    prompt = (
        "請將以下拼音標題轉換為繁體中文，並逐行輸出（每個標題一行）。\n"
        "注意：如果標題包含「=」符號，則僅翻譯「=」前的部分，並且不用加上後面那部分。\n\n"
    )
    prompt += "\n".join(titles_batch)

    response = client.chat.completions.create(
        model="gemini-2.0-flash-exp",
        messages=[{"role": "user", "content": prompt}]
    )

    # Extract the response text
    translated_text = response.choices[0].message.content.strip()
    # Split by newline
    lines = translated_text.split("\n")

    # Clean up whitespace
    translated_titles = [line.strip() for line in lines if line.strip()]

    # Validate length
    if len(translated_titles) != len(titles_batch):
        raise ValueError(
            f"Mismatch between input and translated output lengths.\n"
            f"Input batch size: {len(titles_batch)}\n"
            f"Output lines: {len(translated_titles)}\n"
            f"Full model output:\n{translated_text}"
        )
    return translated_titles

# Process in batches
batch_size = 100  # Adjust as needed
titles = df['title'].tolist()
translated_titles = []

count = 1
for batch in batch_titles(titles, batch_size=batch_size):
    print(f"Starting batch {count}...")
    # We'll retry up to 3 times if a mismatch or other error occurs
    success = False
    for attempt in range(1, 4):
        try:
            batch_result = translate_titles(batch)
            # If translation succeeds, extend the main list
            translated_titles.extend(batch_result)
            success = True
            break  # break from the retry loop
        except Exception as e:
            print(f"Batch {count} translation failed on attempt {attempt}. Error: {e}")
            # Optionally sleep a bit before retrying
            sleep(2)
    
    # If still not successful after 3 attempts, skip this batch
    if not success:
        print("Skipping this batch after 3 failed attempts.")
        # Fill the same number of placeholders (or skip adding them)
        # so that DataFrame rows line up. 
        # If you prefer to leave them blank (NaN), append None or empty strings:
        translated_titles.extend([""] * len(batch))
    
    count += 1
    sleep(5) # Wait a bit between batches (for rate limits, etc.)

df['translated_title'] = translated_titles

# Save
df.to_csv("output/chinese_title_translated.csv", index=False)


Starting batch 1...
Starting batch 2...
Starting batch 3...
Starting batch 4...
Starting batch 5...
Starting batch 6...
Batch 6 translation failed on attempt 1. Error: Mismatch between input and translated output lengths.
Input batch size: 100
Output lines: 93
Full model output:
紅豆粥婆婆
紅湖紀程
紅花草原
紅樓夢
鴻雁向北飛
紅嘴相思鳥的昂貴彩禮
後年的禮物
后羿射日
猴子撈月
小熊的奇節
小蓮與莫內花園
蝴蝶朵朵
虎牛媽媽
花開了
花木蘭
花婆婆
蝸牛的尋遊
划呀划，小木船
花斑狗
花果純真的拋物線
歡迎光臨海愛牛
黃信窗
環境：為啥不應把紙扔在地上？
歡樂的聚會，艾爾菲
會唱歌的貓
會催眠的哈利
回家吧!
會數數的貓
回不去的候車站
灰姑娘進化論
回眸
狐狸爸爸與兒子
狐狸卜
狐狸與樹
虎年的禮物：簡直中國，聽媽媽講老虎的故事，話說銀虎
火裡來，水裡去
火龍
火星歷險記
活了100萬次的貓
火山
火焰
火印
畫給孩子的中國歷史
黑白神玩
河邊的鄰居
狗年的禮物
基礎漢語1000詞
寄居蟹的海洋課
雞媽媽的蛋寶寶
肩上的電影
劍士秦嶼
建築師傑克
殭屍出巡
將軍胡同
驕傲的將軍
交通警察真難當
餃子
教苑魔女
疾病
吉卜林童話
雞蛋裡的悄悄話
揭秘科學
揭秘科學
饑餓遊戲. 1
奇妙的風潮
寂寞商場
金髮姑娘和三隻熊
金髮姑娘和三隻熊
金鬍子和紅毛衣
今天的月亮
精衛填海
經典古文100首
經典連環畫，鮮活英語口語. 1
經典神話故事大全
經典誦讀
精靈與聖誕的秘密
驚險童話
精選神話200篇
鯨魚女孩，池塘男孩
紀念的禮物
金銀島
金銀島
金魚王在吶兒
救命，有海盜！
記憶會記得
將軍與跳蚤
橘子藏哪兒了
巨大的蘿蔔
巨大的蘿蔔
絕地天通
絕對掌控
菊花娃娃
軍艦鳥
Batch 6 translation failed on attempt 2. Error: Mismatch between input and translated output lengths.
Input ba